In [1]:
from pathlib import Path
import itertools
from matplotlib import pyplot as plt
import pandas as pd
import re
from nimstim import create_csv
import numpy as np
from protosc.feature_matrix import FeatureMatrix
from protosc.model.filter import select_with_filter
from protosc.preprocessing import GreyScale, ViolaJones, CutCircle
from protosc.pipe_complex import PipeComplex
from protosc.feature_extraction import FourierFeatures, HOGFeatures
from protosc.io import ReadImage

In [2]:
# Set path to folder with faces: stim_data_dir
stim_data_dir = Path("..", "data", "Nimstim faces")

In [3]:
# Create overview of all images with their corresponding sex, emotion and mouth positioning
overview = create_csv(stim_data_dir)
overview = overview.iloc[:50]

In [4]:
# Define pipeline
pipe1 = ReadImage()*ViolaJones(20)*CutCircle()*FourierFeatures()
pipe2 = ReadImage()*GreyScale()*ViolaJones(20)*CutCircle()*FourierFeatures()
pipe3 = ReadImage()*ViolaJones()*HOGFeatures()
pipe_complex = pipe1 + pipe2 + pipe3

In [5]:
def select_y(df, **kwargs):
    selected = np.ones(len(df), dtype=int)
    for col, val in kwargs.items():
        selected *= (df[col].values == val).astype(int)
    return selected

In [6]:
files = overview['file'].values
y = select_y(overview, mouth="open")
feature_res = pipe_complex.execute(files)

In [7]:
X = FeatureMatrix.from_pipe_data(feature_res)

In [8]:
np.random.seed(1928794)
filter_res = select_with_filter(X, y, fold_seed=1928742)

  0%|          | 0/101 [00:00<?, ?it/s]/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero en

/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r *

/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r *

/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r *

/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r * (df / (1 - r * r))
/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/utils.py:111: RuntimeWarning: divide by zero encountered in true_divide
  ts = r * r *

...

  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 105, in _wrapper_target
    results = exec_func(*args, **kwargs, **job)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 105, in _wrapper_target
    results = exec_func(*args, **kwargs, **job)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 105, in _wrapper_target
    results = exec_func(*args, **kwargs, **job)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 39, in _perform_filter_model
    return model.execute(*args, **kwargs, n_jobs=1)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 39, in _perform_filter_model
    return model.execute(*args, **kwargs, n_jobs=1)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 39, in _perform_filter_model
    return model.execute(*args, **kwargs, n_jobs=1)
  File "/home/qubix

  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 27, in execute
    return execute_parallel(jobs, parallel_model, n_jobs=n_jobs,
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 27, in execute
    return execute_parallel(jobs, parallel_model, n_jobs=n_jobs,
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 27, in execute
    return execute_parallel(jobs, parallel_model, n_jobs=n_jobs,
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 27, in execute
    return execute_parallel(jobs, parallel_model, n_jobs=n_jobs,
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 27, in execute
    return execute_parallel(jobs, parallel_model, n_jobs=n_jobs,
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 27, in execute
    return execute_parallel(jobs, parallel_model, n_jobs=n_jobs

  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in execute_sequential
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in execute_sequential
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in execute_sequential
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in execute_sequential
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in execute_sequential
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in execute_sequential
    results = [targ

  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in <listcomp>
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in <listcomp>
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in <listcomp>
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in <listcomp>
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in <listcomp>
    results = [target(*args, **kwargs, **job) for job in jobs]
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/parallel.py", line 79, in <listcomp>
    results = [target(*args, **kwargs, **job) for job in jobs]
  Fi

  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/base.py", line 41, in parallel_model
    return model._execute_fold(fold)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 28, in _execute_fold
    selected_features, _ = select_features(X_train, y_train)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 28, in _execute_fold
    selected_features, _ = select_features(X_train, y_train)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 28, in _execute_fold
    selected_features, _ = select_features(X_train, y_train)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 28, in _execute_fold
    selected_features, _ = select_features(X_train, y_train)
  File "/home/qubix/Documents/shared_work/protosc/protosc/protosc/model/filter.py", line 28, in _execute_fold
    selected_features, _ = select_features(X_train, y_train)

In [ ]:
X.plot(filter_res)